# Using EcoFOCIpy to process raw field data

## DY1304 ( - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2013/CTDcasts/dy1304/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1304' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1304.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDwWetLabs.yaml'
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd013.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd014.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd015.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd016.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd017.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd024.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd029.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd032.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd033.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd034.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd035.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd037.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd038.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd040.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd041.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd043.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd045.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd046.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd047.btl
Processing /

...

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd082.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd083.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd084.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd085.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd086.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd087.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd088.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd089.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd090.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd091.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd092.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd093.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd094.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd096.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd097.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd098.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd099.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd100.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd102.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd103.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd104.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd105.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd106.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd108.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd109.btl
Processing /Users/bell/ecoraid/2013/CTDcasts/dy1304/rawconverted/ctd110.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd017.btl'].describe()

,sal00,sal11,sbeox0mm/kg,sbeox0ps,sigma-t00,prdm,depsm,t090c,t190c,wetstar,par,cstarat0,cstartr0,v0,v3,v6,scan,datetime
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10
mean,32.482900,32.484990,253.141900,82.687665,25.561480,62.78980,62.192600,6.018420,6.017800,1.697760,1.219603e+02,0.167430,95.949380,0.411860,1.033470,4.525630,16702.800000,2013-04-07 23:42:58.900000256
min,32.146900,32.148700,153.646000,50.926910,25.298000,2.59000,2.566000,5.795100,5.792900,0.120100,1.000000e-12,0.045600,91.602600,0.135100,0.162400,4.320600,9512.000000,2013-04-07 23:37:59
25%,32.157175,32.157975,246.924250,80.405880,25.309650,23.34875,23.131000,5.907100,5.906700,0.253825,1.024605e-02,0.066325,92.452675,0.158525,0.174350,4.360750,13877.000000,2013-04-07 23:41:01
50%,32.277800,32.278250,265.696000,86.692700,25.415550,45.90300,45.472500,5.993100,5.992300,0.412900,5.231250e-01,0.086550,97.859100,0.186450,0.457600,4.615700,17369.000000,2013-04-07 23:43:26.500000
75%,32.550925,32.552450,293.268250,95.570073,25.635650,95.97950,95.068000,6.020300,6.019100,2.640600,1.361870e+01,0.314175,98.357000,0.577275,1.503350,4.639200,19866.000000,2013-04-07 23:45:11
max,33.406100,33.406900,298.203000,97.237890,26.255100,164.10400,162.520000,6.367100,6.367400,5.482900,1.034600e+03,0.350900,98.867300,1.075900,3.463400,4.663200,22095.000000,2013-04-07 23:46:44
std,0.458223,0.460179,51.573364,16.471446,0.344163,53.57161,53.054932,0.181799,0.182221,2.107591,3.247091e+02,0.134438,3.196349,0.369751,1.186353,0.150742,4187.168383,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd017':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1304',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'R/V Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2013, 4, 3),
 'EndDate': datetime.date(2013, 4, 24),
 'Project': 'FOCI, GOA IERP',
 'ChiefScientist': 'Nancy Kachel',
 'StartPort': 'Seward, AK',
 'EndPort': 'Seward, AK',
 'CruiseLocation': '',
 'Description': 'GOA EIRP LTL Survey: CTD, neuston, bongo, (zp & ip), MOCNESS',
 'CruiseYear': 2013,
 'ctdlogs_pdf_name': 'DY1304_CTDLogs.pdf'}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD017']

{'id': 20147,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY1304',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY1304',
 'Project': '',
 'StationNo_altname': '',
 'ConsecutiveCastNo': 'CTD017',
 'LatitudeDeg': 57,
 'LatitudeMin': 56.64,
 'LongitudeDeg': 137,
 'LongitudeMin': 20.6104,
 'GMTDay': 7,
 'GMTMonth': 'apr',
 'GMTYear': 2013,
 'GMTTime': 84660,
 'DryBulb': 6.7,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': 1014,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 143,
 'WindSpd': 16.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 168,
 'StationNameID': 'xst5',
 'MaxDepth': 165,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'Sea-Bird CTD SBE 9',
 'WaterMassCode': 'G',
 'isMooringPr

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd017.btl'] = cruise_data['ctd017.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['ctd017.btl'].sample()

,salinity_ch1,salinity_ch2,sbeox0mm/kg,oxy_percentsat_ch1,sigma_t_ch1,pressure,depsm,temperature_ch1,temperature_ch2,wetstar,par,Attenuation,Transmittance,v0,v3,v6,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
2.0,33.2002,33.2124,170.487,56.50723,26.0844,127.007,125.792,6.3671,6.3674,0.1326,1.000000e-12,0.0456,98.8673,0.1373,0.1635,4.6632,11693.0,2013-04-07 23:39:30,ctd017


In [10]:
cruise_data['ctd017.btl'].columns

Index(['salinity_ch1', 'salinity_ch2', 'sbeox0mm/kg', 'oxy_percentsat_ch1',
       'sigma_t_ch1', 'pressure', 'depsm', 'temperature_ch1',
       'temperature_ch2', 'wetstar', 'par', 'Attenuation', 'Transmittance',
       'v0', 'v3', 'v6', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd017.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0
Data variables: (12/19)
    salinity_ch1        (bottle) float64 80B 33.41 33.2 32.58 ... 32.15 32.15
    salinity_ch2        (bottle) float64 80B 33.41 33.21 32.58 ... 32.15 32.15
    sbeox0mm/kg         (bottle) float64 80B 153.6 170.5 266.6 ... 298.2 297.9
    oxy_percentsat_ch1  (bottle) float64 80B 50.93 56.51 86.8 ... 97.24 97.17
    sigma_t_ch1         (bottle) float64 80B 26.26 26.08 25.67 ... 25.3 25.3
    pressure            (bottle) float64 80B 164.1 127.0 102.4 ... 10.9 2.59
    ...                  ...
    v0                  (bottle) float64 80B 0.1351 0.1373 ... 1.076 0.4296
    v3                  (bottle) float64 80B 0.1624 0.1635 0.1682 ... 2.66 3.463
    v6                  (bottle) float64 80B 4.613 4.663 4.663 ... 4.321 4.323
    scan                (bottle) float64 80B 9.512e+03 1.169e+04 ... 2.21e+04
    datetime            (bottle) datetime64[ns] 80B 2013-04-07T23:37:59 ... 2...
    cast                (bottle) object 80B 'ctd017' 'ctd017' ... 'ctd017'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd017.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 744B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    salinity_ch1        (latitude, longitude, time, bottle) float64 80B 33.41...
    salinity_ch2        (latitude, longitude, time, bottle) float64 80B 33.41...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80B 50.93...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 80B 26.26...
    pressure            (latitude, longitude, time, bottle) float64 80B 164.1...
    temperature_ch1     (latitude, longitude, time, bottle) float64 80B 6.303...
    temperature_ch2     (latitude, longitude, time, bottle) float64 80B 6.303...
    par                 (latitude, longitude, time, bottle) float64 80B 1e-12...

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD017')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 744B
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables:
    salinity_ch1        (latitude, longitude, time, bottle) float64 80B 33.41...
    salinity_ch2        (latitude, longitude, time, bottle) float64 80B 33.41...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80B 50.93...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 80B 26.26...
    pressure            (latitude, longitude, time, bottle) float64 80B 164.1...
    temperature_ch1     (latitude, longitude, time, bottle) float64 80B 6.303...
    temperature_ch2     (latitude, longitude, time, bottle) float64 80B 6.303...
    par                 (latitude, longitude, time, bottle) float64 80B 1e-12...
Attributes: (12/44)
    CruiseID:                      DY1304
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        R/V Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-04-09T14:27:30Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history